<a href="https://colab.research.google.com/github/Anuragse0012/DialogActClassification/blob/master/DialogActClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()
import os

KeyboardInterrupt: ignored

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My Drive

ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive': No such file or directory


In [0]:
import numpy as np
import pandas as pd


from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Add, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras.models import Sequential
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras.callbacks import History
from keras.models import load_model
import keras.backend as K

import matplotlib.pyplot as plt

def trunc_at(s, d, n=3):
    "Returns s truncated at the n'th (3rd by default) occurrence of the delimiter, d."
    return "".join(s.split(d, n)[n:n+1]).strip()    ## return "".join(s.split(d, n)[n-3:n+1]).strip() --> this return full context utterances


MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

# defining list of the classes

label_list = ['%','%--','2','aa','aap','ar','b','ba','bc','bd','bh','bk','br','bs','cc','co','d','fa','ft','g','h','no','qh','qo','qrr','qw','qy','s','t1','t3','x']

#-------loading and preprocessing the training data ---------#

# ----------------------Start--------------------------------------------- #
data_train = pd.read_csv("/content/drive/My Drive/utterances.train", sep='\t', header=None)

trained_Data = []
text_train = data_train[data_train.columns[2]].tolist()


for data in text_train:
    trun_data = trunc_at(data,";")
    trained_Data.append(trun_data)


labels_train = data_train[int(data_train.columns[1])].tolist()

#----------------- End -----------------------------------------------#

#-------loading and preprocessing the development data ---------#

# ----------------------Start--------------------------------------------- #
data_dev = pd.read_csv("/content/drive/My Drive/utterances.valid",sep='\t', header=None)
text_dev = data_dev[data_dev.columns[2]].tolist()

developed_data = []

for data in text_dev:
    trun_data = trunc_at(data,";")
    developed_data.append(trun_data)

labels_dev = data_dev[(data_dev.columns[1])].tolist()

with open('/content/drive/My Drive/developed.txt', 'w') as out_file:
    for i in range(len(developed_data)):
        out_file.write(developed_data[i])
        

        out_file.write('\n')


#----------------- End -----------------------------------------------#

# --- Tokenizing the training data --------------------------------#

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trained_Data)
sequences = tokenizer.texts_to_sequences(trained_Data)
word_index = tokenizer.word_index

print(word_index)
print('Found %s unique tokens.' % len(word_index))

print("word",len(word_index))

word_index['UNK'] = 9754

data_test = pd.read_csv("/content/drive/My Drive/utterances.test", sep='\t',header=None)
#print(data_test.shape)

text_test = data_test[data_test.columns[1]].tolist()

#load the utterances from the test data
utterances_id = data_test[data_test.columns[0]].tolist()

test_data = []

for data in text_test:
    trun_data = trunc_at(data,";")
    test_data.append(trun_data)



sequences_test = []           
for sentence in test_data:
   
    words = sentence.lower().split()
    
    sent = []
    for w in words:
        #print w
        if w not in word_index.keys():
            sent.append(word_index['UNK'])
        else:
            sent.append(word_index[w])
    sequences_test.append(sent)   
    if len(words) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(words)
print("after test",MAX_SEQUENCE_LENGTH)


sequences_dev = []
for sentence in developed_data:
    
    words = sentence.lower().split()
    sent = []
    for w in words:
        if w not in word_index.keys():
            sent.append(word_index['UNK'])
        else:
            sent.append(word_index[w])
    sequences_dev.append(sent)
    
    if len(words) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(words)
print("after dev",MAX_SEQUENCE_LENGTH)
print("length of the sentence",len(sent))




#pad the sequences to the max length
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
data_dev = pad_sequences(sequences_dev, maxlen=MAX_SEQUENCE_LENGTH)


#------ Converting the labels list to binary matrix using encoding ------- #
encoder_train = LabelEncoder()
encoder_train.fit(labels_train)
encoded_trainLabel = encoder_train.transform(labels_train)
labels = to_categorical(np.asarray(encoded_trainLabel))

encoder_dev = LabelEncoder()
encoder_dev.fit(labels_dev)
encoded_devLabel = encoder_dev.transform(labels_dev)
labels_dev = to_categorical(np.asarray(encoded_devLabel))


print('Shape of train_data tensor:', data.shape)
print('Shape of train_label tensor:', labels.shape)

print('Shape of dev_data tensor:', data_dev.shape)
print('Shape of dev_label tensor:', labels_dev.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data

y_train = labels

vocab_size = len(word_index) + 1


#load the whole embedding into the memory
embeddings_index = {}
f = open('/content/drive/My Drive/word2vec_embeddings.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print("loaded %s word vectors. "% len(embeddings_index))

#create a weight matrix for the words in training docs
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

print("embedding range:",embedding_matrix.shape)

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
        
# define model
model = Sequential()
e = Embedding(vocab_size,EMBEDDING_DIM , weights=[embedding_matrix], mask_zero=True, input_length=MAX_SEQUENCE_LENGTH, trainable=False)
model.add(e)
model.add(Dropout(0.50))
model.add(Bidirectional(LSTM(200)))
model.add(Dropout(0.50))

#model.add(Flatten())
model.add(Dense(31, activation='softmax'))

#optimizers : adam, adagrad, sgd, adadelta, adamax, rmsprop

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
model.summary()
# fit the model
history = model.fit(x_train, y_train, validation_data=(data_dev[:8500], labels_dev[:8500]), shuffle=True, batch_size = 128, epochs=4)

#saving the model
model.save("BiDirectionalLSTM-Model-mini-batch_f") 


# evaluate the model
scores = model.evaluate(data_dev[9000:17000], labels_dev[9000:17000], verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

labels_test = model.predict(data_test)

print(type(labels_test))

print(range(len(labels_test)))


with open('/content/drive/My Drive/output.txt', 'w') as out_file:
    for i in range(len(labels_test)):
        out_file.write(utterances_id[i])
        out_file.write('  ')
        out_file.write(label_list[np.argmax(labels_test[i])])

        out_file.write('\n')

Using TensorFlow backend.


{'i': 1, 'and': 2, 'uh': 3, 'the': 4, 'that': 5, 'you': 6, 'it': 7, 'a': 8, 'to': 9, "'s": 10, 'of': 11, 'know': 12, 'yeah': 13, 'they': 14, 'in': 15, 'do': 16, 'huh': 17, "n't": 18, 'we': 19, 'have': 20, 'but': 21, 'so': 22, 'is': 23, 'was': 24, 'like': 25, 'laughter': 26, 'just': 27, 'well': 28, 'think': 29, 'for': 30, 'oh': 31, 'there': 32, 'right': 33, 'or': 34, 'on': 35, 'what': 36, 'really': 37, "'re": 38, 'not': 39, 'are': 40, 'my': 41, 'um': 42, 'be': 43, 'with': 44, 'if': 45, 'one': 46, 'he': 47, 'about': 48, "'m": 49, 'all': 50, 'get': 51, 'because': 52, "'ve": 53, 'had': 54, 'would': 55, 'out': 56, 'them': 57, 'at': 58, 'as': 59, 'when': 60, 'lot': 61, 'this': 62, 'up': 63, 'people': 64, 'some': 65, 'did': 66, 'then': 67, 'go': 68, 'going': 69, 'can': 70, 'mean': 71, 'no': 72, 'kind': 73, 'now': 74, 'good': 75, 'time': 76, 'got': 77, 'things': 78, 'she': 79, 'were': 80, 'more': 81, 'from': 82, 'see': 83, 'too': 84, 'something': 85, 'been': 86, 'me': 87, 'how': 88, 'very': 89

In [0]:
import pandas as pd

data_train = pd.read_csv("/content/drive/My Drive/utterances.valid", sep='\t', header=None)

trained_Data = []
text_train = data_train[data_train.columns[2]].tolist()

for data in text_train:
    trun_data = trunc_at(data,";")
    trained_Data.append(trun_data)

with open('/content/drive/My Drive/train.txt', 'w') as out_file:
    for i in range(len(trained_Data)):
        out_file.write(trained_Data[i])
        
        out_file.write('\n')



In [0]:
s = "Line - out into the line - in on a tape player , ; Mm - hmm . ; Hmm . ; Yep ."
s1 = ";  ;  ; Okay , uh"
def trunc_at(s, d, n=3):
    "Returns s truncated at the n'th (3rd by default) occurrence of the delimiter, d."
    
    return "".join(s.split(d, n)[n-3:n+1]).strip()

print(trunc_at(s,";"))

In [0]:
import numpy as np

t = [[4,5,6,7,1,8,4],[6,7,8,9,6,7,8]]

print(type(t))

for i in range(len(t)):
  print(np.argmax(t[i]))

In [0]:
import pandas as pd

data_train = pd.read_csv("/content/drive/My Drive/utterances.train", sep='\t', header=None)

trained_Data = []
text_train = data_train[data_train.columns[2]].tolist()

def trunc_at(s, d, n=3):
    "Returns s truncated at the n'th (3rd by default) occurrence of the delimiter, d."
    return "".join(s.split(d, n)[n:n+1]).strip()  ## return "".join(s.split(d, n)[n-3:n+1]).strip() --> this return full context utterances

for data in text_train[:50]:
    trun_data = trunc_at(data,";")
    trained_Data.append(trun_data)
    
    
for i in range(len(trained_Data)):
  print(trained_Data[i])
